In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.5/882.5 kB 18.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

# Carregar o modelo de temperatura treinado
temperature_model = tf.keras.models.load_model('/content/drive/MyDrive/temperature_digits/temperature_model.h5')


In [ ]:
import numpy as np
# Function to preprocess the digit image
def preprocess_digit(digit):
    digit_resized = cv2.resize(digit, (16, 21))
    digit_normalized = digit_resized.astype('float32') / 255.0
    digit_ready = np.expand_dims(np.expand_dims(digit_normalized, axis=0), axis=-1)
    return digit_ready

# Function to recognize the digit using the CNN model
def recognize_digit(digit):
    preprocessed_digit = preprocess_digit(digit)
    prediction = temperature_model.predict(preprocessed_digit, verbose=0)
    predicted_class = np.argmax(prediction)

    class_mapping = {
        0: '0', 1: '1', 2: '2', 3: '3', 4: '4',
        5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
        10: 'minus', 11: 'nothing'
    }

    return class_mapping.get(predicted_class, 'unknown')

# Function to extract temperature from a rectangular region
def extract_temperature(image, x1, y1, x2, y2):
    roi = image[y1:y2, x1:x2]

    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    digit1 = gray_roi[:, :16]
    digit2 = gray_roi[:, 16:32]
    digit3 = gray_roi[:, 48:64]

    recognized_digit1 = recognize_digit(digit1)
    recognized_digit2 = recognize_digit(digit2)
    recognized_digit3 = recognize_digit(digit3)

    if recognized_digit1 == 'minus':
        recognized_number = f"-{recognized_digit2}.{recognized_digit3}"
    else:
        recognized_number = f"{recognized_digit1}{recognized_digit2}.{recognized_digit3}"

    #print(f"Recognized temperature: {recognized_number}")

    try:
        return float(recognized_number)
    except ValueError:
        return None

# Function to process the thermal image and compute the final robust temperature
def process_thermal_image(img, x1, y1, x2, y2):
    if img is None:
        return None

    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    height, width = img.shape[:2]

    # Clampear as coordenadas
    x1 = max(0, min(x1, width - 1))
    y1 = max(0, min(y1, height - 1))
    x2 = max(0, min(x2, width - 1))
    y2 = max(0, min(y2, height - 1))

    # Extrair temperaturas de referência
    temp_high = extract_temperature(img, 510, 67, 575, 88)
    temp_low = extract_temperature(img, 510, 403, 575, 424)

    if temp_high is None or temp_low is None:
        return None

    roi = img[y1:y2, x1:x2, 0].astype(np.float32)

    temp_range = temp_high - temp_low
    pixel_temps = temp_low + (roi / 255.0 * temp_range)

    sorted_temps = np.sort(pixel_temps.flatten())
    threshold_index = int(0.9 * sorted_temps.size)
    final_temp = np.mean(sorted_temps[threshold_index:])

    if final_temp == None:
        final_temp = 0.0

    return final_temp

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Paths
model_path = '/content/drive/MyDrive/yolo/best (2).pt'
input_video_path = '/content/drive/MyDrive/yolo/00000000220000100 (online-video-cutter.com).mp4'
output_video_path = '/content/output_video.mp4'

In [17]:
from collections import defaultdict

model = YOLO(model_path)

# Definir as classes
CLASS_HEAD = 0
CLASS_EYE = 1

# Inicializar conjuntos para armazenar IDs únicos e contadores
head_ids = set()
eye_ids = set()
eye_count = defaultdict(int)  # Dicionário para contar ocorrências dos IDs dos olhos
eye_temperatures = {}  # Armazena a última temperatura calculada para cada ID de olho

# Usar o rastreador integrado do YOLOv8 para processar o vídeo
results = model.track(source=input_video_path, conf=0.2, persist=True, save=False)

# Abrir o vídeo de entrada para obter informações de dimensão
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()

# Definir a saída do vídeo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

for result in results:
    frame = result.orig_img  # Imagem original
    boxes = result.boxes

    for box in boxes:
        # Verificar se o ID está disponível
        if box.id is None:
            continue  # Ignorar se não houver ID (objeto não rastreado)

        # Extrair as informações do track
        bbox = box.xyxy[0].cpu().numpy().astype(int)
        x1, y1, x2, y2 = bbox
        conf = box.conf[0].cpu().numpy()
        cls = int(box.cls[0].cpu().numpy())
        track_id = int(box.id[0].cpu().numpy())

        # Adicionar o ID ao conjunto correspondente
        if cls == CLASS_HEAD:
            head_ids.add(track_id)
            color = (0, 255, 255)  # Amarelo para "cabeça"
            label = f'Cabeca ID {track_id}'
        elif cls == CLASS_EYE:
            eye_count[track_id] += 1  # Incrementar o contador para o ID do olho

            # Verificar se a temperatura já foi calculada ou precisa ser recalculada
            if track_id not in eye_temperatures or eye_count[track_id] >= 10:
                # Recalcular a temperatura após 10 ocorrências
                eye_temp = process_thermal_image(frame, x1, y1, x2, y2)
                eye_temperatures[track_id] = eye_temp  # Armazenar a nova temperatura
                eye_count[track_id] = 0  # Resetar o contador após recalcular
            else:
                # Usar a temperatura já calculada
                eye_temp = eye_temperatures[track_id]

            eye_ids.add(track_id)
            color = (255, 0, 255)  # Magenta para "olho"

            if eye_temp is not None:
                label = f'Olho ID {track_id}, {eye_temp:.2f}'
            else:
                label = f'Olho ID {track_id}, temperatura nao disponivel'
        else:
            color = (0, 255, 0)  # Verde para outras classes
            label = f'ID {track_id}'

        # Desenhar a caixa delimitadora e o rótulo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Escrever o frame processado no vídeo de saída
    out.write(frame)

# Obter as contagens finais
total_cabecas = len(head_ids)
total_olhos = len(eye_ids)
print(f'Total de cabeças rastreadas: {total_cabecas}')
print(f'Total de olhos rastreados: {total_olhos}')

# Limpeza
out.release()
cv2.destroyAllWindows()




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2998) /content/drive/MyDrive/yolo/00000000220000100 (online-video-cutter.com).mp4: 512x640 1 cabeca, 11.4ms
video 1/1 (frame 2/2998) /content/drive/MyDrive/yolo/00000000220000100 (online-video-cutter.com).mp4: 512x640 1 cabeca, 8.8ms
video 1/1 (frame 3/2998) /content/drive/MyDrive/yolo/00000000220000100 (online-video-cutter.com).mp4: 512x640 1 cabeca, 8.6ms
video 1/1 (frame 4/2998) /content/drive/MyDrive/yolo/00000000220000100 (onlin

In [ ]:
output_video_path = '/content/output_video_no_tracking.mp4'
model = YOLO(model_path)


# Defina os IDs das classes conforme seu modelo YOLOv8
CLASS_HEAD = 0
CLASS_EYE = 1

# Inicialize conjuntos para armazenar IDs únicos de cabeças e olhos
head_ids = set()
eye_ids = set()


cap = cv2.VideoCapture(input_video_path)

# Defina a saída do vídeo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 25.0, (int(cap.get(3)), int(cap.get(4))))


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Use o modelo YOLOv8 para fazer inferências no frame
    results = model(frame)

    # Processar os resultados do modelo YOLOv8
    for box in results[0].boxes:
        # Extrair as coordenadas da caixa delimitadora
        bbox_array = box.xyxy.cpu().numpy().flatten()
        x1, y1, x2, y2 = bbox_array.astype(int)
        w = x2 - x1
        h = y2 - y1

        # Extrair a confiança e a classe
        conf = box.conf.cpu().numpy()[0]
        cls = int(box.cls.cpu().numpy()[0])

        if conf > 0.2:  # Filtrar por confiança
            # Desenhar a caixa delimitadora e o rótulo diretamente no frame
            if cls == CLASS_HEAD:
                color = (0, 255, 255)  # Amarelo para "cabeça"
                label = f'Cabeca: {conf:.2f}'
            elif cls == CLASS_EYE:
                color = (255, 0, 255)  # Magenta para "olho"
                label = f'Olho: {conf:.2f}'
            else:
                color = (0, 255, 0)  # Verde para outras classes
                label = f'Classe {cls}: {conf:.2f}'

            # Desenhar a caixa delimitadora
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Desenhar o rótulo
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Escreve o frame processado no vídeo de saída
    out.write(frame)

    # Verificar se a tecla 'q' foi pressionada para sair
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Limpeza
cap.release()
out.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
0: 512x640 2 cabecas, 3 olhos, 9.4ms
Speed: 1.8ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 2 cabecas, 3 olhos, 8.9ms
Speed: 1.8ms preprocess, 8.9ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 2 cabecas, 3 olhos, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 2 cabecas, 3 olhos, 11.3ms
Speed: 1.7ms preprocess, 11.3ms inference, 1.8ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 cabecas, 3 olhos, 8.7ms
Speed: 1.7ms preprocess, 8.7ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 cabecas, 3 olhos, 11.5ms
Speed: 2.2ms preprocess, 11.5ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 cabecas, 3 olhos, 8.8ms
Speed: 1.9ms preprocess, 8.8ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)

0: 51